In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import os
import cv2
import numpy as np


In [2]:
class UNet(nn.Module):
  def __init__(self):
    super(UNet,self).__init__()

    self.downsample = nn.Sequential(

        nn.Conv2d(3, 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.MaxPool2d(2),

        nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.MaxPool2d(2),

        nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.MaxPool2d(2),

    )

    self.upsample = nn.Sequential(

        nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True),
        nn.Conv2d(256, 128, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True),
        nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(negative_slope=0.01, inplace = True),
        nn.MaxPool2d(2),

        nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True),
        nn.Conv2d(128, 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(negative_slope=0.01, inplace= True),
        nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True),
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(negative_slope = 0.01, inplace = True),
        nn.MaxPool2d(2),

        nn.Upsample(scale_factor = 2, mode = 'bilinear', align_corners = True),
        nn.Conv2d(64, 3, kernel_size = 3, padding = 1),
        nn.ReLU()
    )

  def forward(self,x):
    ds = self.downsample(x)
    output = self.upsample(ds)

    return output

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:

model1 = UNet()
model_path = '20_unet_ploss_vgg19.pth'
model1.load_state_dict(torch.load(model_path, map_location=device))
model1 = model1.to(device)
model1.eval()


UNet(
  (downsample): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01, inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.01, inplace=True)
   

In [10]:

input_path = "images/low"
output_path = "images/high"

if not os.path.exists(output_path):
    os.makedirs(output_path)

all_files = os.listdir(input_path)

for image_name in all_files:
    image_path = os.path.join(input_path, image_name)
    image = cv2.imread(image_path)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    target_size = (128, 128)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

    image = image.astype(np.float32) / 255.0
    image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).to(device)
    
    with torch.no_grad():
        predicted = model1(image).squeeze(0).cpu().numpy()
    
    predicted = predicted.transpose(1, 2, 0)
    predicted = (predicted * 255).astype(np.uint8)

    output_image_path = os.path.join(output_path, image_name)
    cv2.imwrite(output_image_path, cv2.cvtColor(predicted, cv2.COLOR_RGB2BGR))


In [11]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def calculate_psnr(directory_pred, directory_gt):
    psnr_values = []
    for filename_pred in os.listdir(directory_pred):
        if filename_pred.endswith('.jpg') or filename_pred.endswith('.png'):
            filename_gt = os.path.join(directory_gt, filename_pred)
            filename_pred = os.path.join(directory_pred, filename_pred)

            img_pred = cv2.imread(filename_pred)
            img_gt = cv2.imread(filename_gt)

            if img_pred is not None and img_gt is not None:
                psnr_values.append(psnr(img_gt, img_pred))
            else:
                print(f"Could not read images: {filename_gt}, {filename_pred}")

    avg_psnr = np.mean(psnr_values)
    return avg_psnr

directory_pred = "images/high"
directory_gt = "images/ground_truth"

avg_psnr = calculate_psnr(directory_pred, directory_gt)
print("Average PSNR:", avg_psnr)

Average PSNR: 27.997436901539505


In [12]:

model2 = UNet()
model_path = '15_unet_ploss_vgg19.pth'
model2.load_state_dict(torch.load(model_path, map_location=device))
model2 = model1.to(device)
model2.eval()


UNet(
  (downsample): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01, inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): LeakyReLU(negative_slope=0.01, inplace=True)
   

In [13]:

input_path = "images/low"
output_path = "images/high"

if not os.path.exists(output_path):
    os.makedirs(output_path)

all_files = os.listdir(input_path)

for image_name in all_files:
    image_path = os.path.join(input_path, image_name)
    image = cv2.imread(image_path)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    target_size = (128, 128)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)

    image = image.astype(np.float32) / 255.0
    image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).to(device)
    
    with torch.no_grad():
        predicted = model2(image).squeeze(0).cpu().numpy()
    
    predicted = predicted.transpose(1, 2, 0)
    predicted = (predicted * 255).astype(np.uint8)

    output_image_path = os.path.join(output_path, image_name)
    cv2.imwrite(output_image_path, cv2.cvtColor(predicted, cv2.COLOR_RGB2BGR))


In [14]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def calculate_psnr(directory_pred, directory_gt):
    psnr_values = []
    for filename_pred in os.listdir(directory_pred):
        if filename_pred.endswith('.jpg') or filename_pred.endswith('.png'):
            filename_gt = os.path.join(directory_gt, filename_pred)
            filename_pred = os.path.join(directory_pred, filename_pred)

            img_pred = cv2.imread(filename_pred)
            img_gt = cv2.imread(filename_gt)

            if img_pred is not None and img_gt is not None:
                psnr_values.append(psnr(img_gt, img_pred))
            else:
                print(f"Could not read images: {filename_gt}, {filename_pred}")

    avg_psnr = np.mean(psnr_values)
    return avg_psnr

directory_pred = "images/high"
directory_gt = "images/ground_truth"

avg_psnr = calculate_psnr(directory_pred, directory_gt)
print("Average PSNR:", avg_psnr)

Average PSNR: 27.997436901539505


In [ ]:
import os

def delete_files_in_directory(directory):
    file_list = os.listdir(directory)
    
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted file: {file_path}")

directory_path = "/Users/ayushsingh/Desktop/Image-Denoising/experiments-and-trials/images/high"

delete_files_in_directory(directory_path)